In [ ]:
# Import necessary libraries and frameworks

import numpy as np
import random
from sklearn.model_selection import train_test_split
import pandas as pd
import os
from tensorflow.python.client import device_lib
from sklearn.metrics import *
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import tensorflow as tf
import time

In [ ]:
# Load datasets for different modalities (text, audio, vision, and game) for training and testing

train_target = np.load('path_to_data/train_target.npy')
train_text = np.load('path_to_data/train_text.npy')
train_audio = np.load('path_to_data/train_audio.npy')
train_vision = np.load('path_to_data/train_vision.npy')
train_game = np.load('path_to_data/train_game.npy')

test_target = np.load('path_to_data/test_target.npy')
test_text = np.load('path_to_data/test_text.npy')
test_audio = np.load('path_to_data/test_audio.npy')
test_vision = np.load('path_to_data/test_vision.npy')
test_game = np.load('path_to_data/test_game.npy')

In [ ]:
# Prepare dataset for training
# This includes caching, shuffling, batching, and prefetching to optimize data loading during training

BATCH_SIZE = 64
BUFFER_SIZE = 20000

train_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'text_input': train_text,
        'audio_input': train_audio,
        'vision_input': train_vision,
        'game_input': train_game
    },
    {
        'outputs': train_target  
    },
))

train_dataset = train_dataset.cache()
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
# Prepare  dataset for testing
# This includes caching, batching, and prefetching to optimize data loading during evaluation

BATCH_SIZE = 64
BUFFER_SIZE = 20000

test_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'text_input': test_text,
        'audio_input': test_audio,
        'vision_input': test_vision,
        'game_input': test_game
    },
    {
        'outputs': test_target  
    },
))

test_dataset = test_dataset.cache()
test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
# Define hyperparameters for the model and training process

text_size = train_text.shape[1]           # Text Vector length
audio_size = train_audio.shape[1]         # Audio Vector length
vision_size = train_vision.shape[1]       # Vision Vector length
game_size = train_game.shape[1]           # Game Vector length


text_dim = train_text.shape[2]
audio_dim = train_audio.shape[2]
vision_dim = train_vision.shape[2]
game_dim = train_game.shape[2]


kernel_size = 1
num_layers = 3        # Number of layers in the encoder and decoder of the model
dff = 128             # Hidden layer size in feed-forward network
d_model = 32          # Input and output size of the transformer's encoder and decoder
num_heads = 8         # Number of parallel attention heads
dropout = 0.1         # Dropout rate

lr_rate = 0.00005     # Learning rate
neg, pos = np.bincount(train_target.astype(np.int64))
total = neg + pos
output_bias = tf.keras.initializers.Constant(np.log([pos/neg]))

In [ ]:
from Working_code.model_architecture import overall
from sklearn.utils import class_weight

model = overall(
    text_size = text_size,
    audio_size = audio_size,
    vision_size = vision_size,
    game_size = game_size,
    text_dim = text_dim,
    audio_dim = audio_dim,
    vision_dim = vision_dim,
    game_dim = game_dim,
    kernel_size = kernel_size,
    num_layers = num_layers,
    dff = dff,
    d_model=d_model,
    num_heads = num_heads,
    dropout = dropout,
    output_bias = output_bias)

METRICS = [
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.AUC(name='auc')
]

class_weight = class_weight.compute_class_weight('balanced', classes=np.unique(train_target), 
                                                  y=train_target)

focal_loss = tf.keras.losses.BinaryFocalCrossentropy(
    apply_class_balancing=True, alpha=(class_weight[1]/class_weight[0]), gamma=2)

model.compile(loss=focal_loss, 
              optimizer=tf.keras.optimizers.Adam(lr_rate), metrics=METRICS)
model.summary()

In [ ]:
# Visualize the training and validation accuracy and loss over epochs
# Note: Validation curves are generally smoother than training curves.
# This is mainly because dropout layers are not activated during evaluation.

name = 'ours'
checkpoint_path = "saved_model/" + types + ".h5"

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_accuracy', 
                                                   mode='max', save_best_only=True, save_weights_only=True)
                                                  
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss', mode='min')

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(patience=2, monitor='val_loss', factor=0.5)

history = model.fit(train_dataset, epochs=100,
                    validation_data = test_dataset,
                    callbacks = [checkpoint_cb, early_stopping_cb, reduce_lr])

In [ ]:
# Evaluate the model's performance on the test dataset
# Predict the test dataset
# Evaluate the predictions using metrics like accuracy, F1 score, and AUC

epochs = range(1, len(history.history['accuracy']) + 1)
plt.plot(epochs, history.history['accuracy'])
plt.plot(epochs, history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epochs')
plt.legend(['train', 'valid'], loc='lower right')
plt.ylim([0.7, 1])   
plt.show()

epochs = range(1, len(history.history['loss']) + 1)
plt.plot(epochs, history.history['loss'])
plt.plot(epochs, history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(['train', 'valid'], loc='upper right')
plt.show()